<p style="text-align:center;">
<img src="https://github.com/digital-futures-academy/DataScienceMasterResources/blob/main/Resources/datascience-notebook-header.png?raw=true"
     alt="DigitalFuturesLogo"
     style="float: center; margin-right: 10px;" />
</p>

## Digital Futures Data Science Programme
### Linear Regression Team Project - Results Template
#### V1

## Objective:
#### Create pricing function. To allow all 7 projects to be combined, you must stick to this template. 

In [1]:
## MAKE 

car_input = {'make': 'xyz',
             'model':'xyz',
             'year':'xyz',
             'writeoff':'xyz',
             'mileage':'xyz',
             'BHP':'xyz',
             'transmission':'xyz',
             'fuel':'xyz',
             'owners':'xyz',
             'body':'xyz',
             'ULEZ':'xyz',
             'engine':'xyz',
             'Condition':'xyz',
            }


In [2]:
import pandas as pd
import numpy as np

In [3]:
def suggest_price__Audi(car_input):

    """Converting the inputted dictionary into a pandas dataframe, and then applying feature engineering"""
    car_df = pd.DataFrame([car_input])


    def feature_eng(data):

        """Creating a copy of the inputted dataframe"""
        result = data.copy()

        """Formatting the columns from strings to their proper type"""
        result['year'] = result['year'].astype(int)
        result['mileage'] = result['mileage'].astype(float)
        result['BHP'] = result['BHP'].astype(float)
        result['owners'] = result['owners'].astype(int)
        result['engine'] = result['engine'].replace('L', '', regex=True).astype(float)

        """Dropping unneeded columns"""
        result.drop(columns=['ULEZ','make', 'model', 'Condition','writeoff'], inplace=True)

        """Adding new columns"""
        result['log_mileage'] = np.log(result['mileage'])
        result['log_owners'] = np.log(result['owners'])

        result = pd.get_dummies(result, columns=['transmission'], drop_first=False, prefix='gear_type')
        result = pd.get_dummies(result, columns=['fuel'], drop_first=False, prefix='fuel_type')
        result = pd.get_dummies(result, columns=['body'], drop_first=False, prefix='body_type')

        result['const'] = 1.0

        return result

    car_df_feat = feature_eng(car_df)

    """Setting the price modifier based on the damage of the car"""
    modifier = 0.8
    if car_input['Condition'] == 'Moderate damage':
        modifier = 0.5
    elif car_input['Condition'] == 'Heavily damage':
        modifier = 0.3

    """Constructing a dictionary of coefficients for the various features in our model"""
    parameters = {'year'                  : 1.134488e+03,
                  'const'                 :-1.499547e+06,
                  'BHP'                   : 4.208970e+01,
                  'log_mileage'           :-2.841232e+03,
                  'body_type_SUV'         : 2.398625e+03,
                  'gear_type_Automatic'   :-7.488307e+05,
                  'gear_type_Manual'      :-7.507164e+05,
                  'engine'                : 1.585720e+03,
                  'body_type_Estate'      : 8.165405e+02,
                  'mileage'               :-1.500031e-02,
                  'body_type_Coupe'       :-4.578385e+02,

                  'owners'                           : 0,
                  'log_owners'                       : 0,
                  'fuel_type_Diesel'                 : 0,
                  'fuel_type_Petrol'                 : 0,
                  'fuel_type_Petrol Plug-in Hybrid'  : 0,
                  'body_type_Convertible'            : 0,
                  'body_type_Hatchback'              : 0,
                  'body_type_Saloon'                 : 0,
                  }

    """Setting the price of the car as equal to £0"""
    base_price = 0
    """Updating the price by looping through the columns in our dataframe, and multiplying those values by the coefficients in the parameters dictionary"""
    for column in car_df_feat.columns:
        base_price += parameters[column] * car_df_feat.loc[0, column]

    "Calculating our suggested price, based on the damage to the vehicle"
    Suggested_Price = base_price * modifier

    if Suggested_Price <= 0:
        return f"We have valued your car at £{Suggested_Price}. As such, we are not willing to pay for it."

    return Suggested_Price